In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

In [13]:
!kaggle datasets download -d saidakbarp/17-category-flowers

 85% 49.0M/57.5M [00:01<00:00, 25.1MB/s]
100% 57.5M/57.5M [00:01<00:00, 45.7MB/s]


In [14]:
!unzip -qq 17-category-flowers.zip

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam

In [2]:
folder_address = '/content/drive/MyDrive/Datasets/17flowers'

In [3]:
width = height = 224
batch_size = 8

In [4]:
data_generator = ImageDataGenerator(
    rescale = 1 / 255,
    validation_split = 0.2,
)

train_data = data_generator.flow_from_directory(folder_address,
                                                target_size = (width, height),
                                                class_mode = 'categorical',
                                                batch_size = batch_size,
                                                shuffle = True,
                                                subset='training')

val_data = data_generator.flow_from_directory(folder_address,
                                              target_size = (width, height),
                                              class_mode = 'categorical',
                                              batch_size = batch_size,
                                              shuffle = True,
                                              subset='validation')

Found 1088 images belonging to 17 classes.
Found 272 images belonging to 17 classes.


In [5]:
base_model = tf.keras.applications.ResNet50V2(
    input_shape=(width, height, 3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'max'
    )

94683136/94668760 [==============================] - 1s 0us/step


In [6]:
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [7]:
model = tf.keras.models.Sequential([
    base_model,
    Dense(17, activation='softmax')                          
])

In [8]:
model.compile(optimizer = tf.keras.optimizers.SGD(),
              loss = tf.keras.losses.binary_crossentropy,
              metrics='accuracy')

In [11]:
model.fit(train_data,
          steps_per_epoch = train_data.samples / batch_size,
          validation_data = val_data,
          validation_steps = val_data.samples / batch_size,
          epochs = 20
)

Epoch 1/20
136/136 [==============================] - 370s 2s/step - loss: 0.3147 - accuracy: 0.5129 - val_loss: 0.2519 - val_accuracy: 0.6544
Epoch 2/20
136/136 [==============================] - 31s 230ms/step - loss: 0.0616 - accuracy: 0.8879 - val_loss: 0.0886 - val_accuracy: 0.8493
Epoch 3/20
136/136 [==============================] - 32s 231ms/step - loss: 0.0295 - accuracy: 0.9614 - val_loss: 0.0848 - val_accuracy: 0.8640
Epoch 4/20
136/136 [==============================] - 32s 232ms/step - loss: 0.0168 - accuracy: 0.9844 - val_loss: 0.0799 - val_accuracy: 0.8787
Epoch 5/20
136/136 [==============================] - 32s 232ms/step - loss: 0.0111 - accuracy: 0.9908 - val_loss: 0.0677 - val_accuracy: 0.8676
Epoch 6/20
136/136 [==============================] - 32s 233ms/step - loss: 0.0101 - accuracy: 0.9936 - val_loss: 0.0794 - val_accuracy: 0.8603
Epoch 7/20
136/136 [==============================] - 32s 232ms/step - loss: 0.0069 - accuracy: 0.9926 - val_loss: 0.0652 - val_accu